In [1]:
  import pandas as pd
  import numpy as np
  import plotly.express as px
  import plotly.graph_objects as go
  import folium
  from folium.plugins import HeatMap
  from folium.plugins import MarkerCluster
  import requests
  import random

In [2]:
est_url = 'https://raw.githubusercontent.com/angelnmara/geojson/master/mexicoHigh.json' 
mun_url = 'https://raw.githubusercontent.com/angelnmara/geojson/master/MunicipiosMexico.json'
# Archivo GeoJSON
est_regions = requests.get(est_url).json()
mun_regions = requests.get(mun_url).json()

In [3]:
# función 
def make_dataframe_states(geo_data):
  '''
  Función para obtener un dataframe de estados
  '''
  # get features
  features = geo_data['features']
  # get country, state and mun
  country = []
  state = []

  # loop for each feature
  for feature in features:
    country.append(feature['properties']['CNTRY'])
    state.append(feature['properties']['name'])
    #mun.append(feature['properties']['NAME_2'])
  # dataframe
  df2 = pd.DataFrame([country, state]).T
  df2.columns = ['country', 'state']
  # random percentage
  df2['percentage'] = [random.uniform(0, 1) for i in range(df2.shape[0])]

  return df2

In [4]:
# función 
def make_dataframe_mun(geo_data):
  '''
  Función para obtener un dataframe de estados y municipios 
  '''
  # get features
  features = geo_data['features']
  # get country, state and mun
  country = []
  state = []
  mun = []
  # loop for each feature
  for feature in features:
    country.append(feature['properties']['NAME_0'])
    state.append(feature['properties']['NAME_1'])
    mun.append(feature['properties']['NAME_2'])
  # dataframe
  df2 = pd.DataFrame([country, state, mun]).T
  df2.columns = ['country', 'state', 'mun']
  # random percentage
  df2['percentage'] = [random.uniform(0, 1) for i in range(df2.shape[0])]
  
  return df2

In [95]:
corrected_states = {'Aguascalientes': 'Aguascalientes',
                    'Baja California': 'Baja California',
                    'Baja California Sur': 'Baja California Sur',
                    'Campeche': 'Campeche',
                    'Chiapas': 'Chiapas',
                    'Chihuahua': 'Chihuahua', 
                    'Ciudad de México': 'Ciudad de México',
                    'Colima': 'Colima',
                    'Durango': 'Durango',
                    'Guanajuato':'Guanajuato',
                    'Guerrero': 'Guerrero',
                    'Hidalgo': 'Hidalgo',
                    'Jalisco': 'Jalisco',
                    'México': 'México',
                    'Morelos': 'Morelos',
                    'Nayarit': 'Nayarit',
                    'Nuevo León': 'Nuevo León',
                    'Oaxaca': 'Oaxaca',
                    'Puebla': 'Puebla',
                    'Querétaro': 'Querétaro',
                    'Quintana Roo': 'Quintana Roo', 
                    'San Luis Potosí': 'San Luis Potosí',
                    'Sinaloa': 'Sinaloa',
                    'Sonora': 'Sonora',
                    'Tabasco': 'Tabasco',
                    'Tamaulipas': 'Tamaulipas',
                    'Tlaxcala': 'Tlaxcala',
                    'Yucatán': 'Yucatán',
                    'Zacatecas': 'Zacatecas',
                    'Coahuila': 'Coahuila de Zaragoza',
                    'Veracruz': 'Veracruz de Ignacio de la Llave',
                    'Michoacán': 'Michoacán de Ocampo'}


df_est = make_dataframe_states(est_regions)
df_est['state2'] = df_est['state'].map(corrected_states)
df_est = df_est.drop('state', axis=1)
df_est = df_est.rename(columns={'state2':'state'})
df_est.head()

,country,percentage,state
0,Mexico,0.462432,Zacatecas
1,Mexico,0.356434,Yucatán
2,Mexico,0.771410,Veracruz de Ignacio de la Llave
3,Mexico,0.222384,Tlaxcala
4,Mexico,0.286977,Tamaulipas


In [6]:
df_mun = make_dataframe_mun(mun_regions)
df_mun.head()

,country,state,mun,percentage
0,Mexico,Aguascalientes,Aguascalientes,0.375065
1,Mexico,Aguascalientes,Asientos,0.584549
2,Mexico,Aguascalientes,Calvillo,0.475717
3,Mexico,Aguascalientes,Cosío,0.519900
4,Mexico,Aguascalientes,Jesús María,0.728594


# **Bases producción**

In [98]:
# Base INEGI
colnames_inegi = ['CVE_ENT','NOM_ENT','CVE_MUN','NOM_MUN']
df_inegi = pd.read_excel('../datasets/AGEEML_2023311323291.xlsx')
df_inegi = df_inegi[colnames_inegi]

# nombres de columnas
colnames = ['Anio', 'Idestado', 'Nomestado', 'Idddr', 'Nomddr', 'Idcader',
       'Nomcader', 'Idmunicipio', 'Nommunicipio', 'Idciclo',
       'Nomcicloproductivo', 'Idmodalidad', 'Nommodalidad', 'Idunidadmedida',
       'Nomunidad', 'Idcultivo', 'Nomcultivo', 'Sembrada', 'Cosechada',
       'Siniestrada', 'Volumenproduccion', 'Rendimiento', 'Precio',
       'Valorproduccion']
selected_columns = ['Anio', 'Idestado', 'Nomestado', 'Idmunicipio',	'Nommunicipio', 'Nomunidad', 'Nomcultivo', 'Volumenproduccion']
# leemos bases y cambiamos nombres de columnas para homologar nombres    
#  base 2019   
df_prod_2019 = pd.read_csv('../datasets/Cierre_agricola_mun_2019.csv',  encoding = "ISO-8859-1")
df_prod_2019.columns = colnames
df_prod_2019 = df_prod_2019[selected_columns]
df_prod_2019['Anio'] = df_prod_2019['Anio'].astype('int')
#  base 2020
df_prod_2020 = pd.read_csv('../datasets/Cierre_agricola_mun_2020.csv',  encoding = "ISO-8859-1")
df_prod_2020.columns = colnames
df_prod_2020 = df_prod_2020[selected_columns]
#  base 2021
df_prod_2021 = pd.read_csv('../datasets/Cierre_agr_mun_2021.csv',  encoding = "ISO-8859-1")
df_prod_2021.columns = colnames
df_prod_2021 = df_prod_2021[selected_columns]
# producción de leche 2021
df_prod_leche = pd.read_excel('../datasets/ProdLeche.xlsx')
df_prod_leche = df_prod_leche.drop('EML', axis=1)
df_prod_leche['Idestado'] = np.nan
df_prod_leche['Idmunicipio'] = np.nan
df_prod_leche['Nomunidad'] = 'Miles de lts'
df_prod_leche['Nomcultivo'] = 'Leche'
df_prod_leche = df_prod_leche[['Año', 'Idestado', 'Estado', 'Idmunicipio',	'Municipio', 'Nomunidad', 'Nomcultivo', 'Prod (Miles de lts)']]
df_prod_leche.columns = selected_columns

En este paso se unen las bases por filas 

In [99]:
# bases frijol arraoz, trigo ...
base_2019 = pd.merge(df_inegi, df_prod_2019, 
                     left_on=['CVE_ENT', 'CVE_MUN'], 
                     right_on=['Idestado','Idmunicipio'], how='left')
base_2019['Anio'] = 2019
base_2020 = pd.merge(df_inegi, df_prod_2020, 
                     left_on=['CVE_ENT', 'CVE_MUN'], 
                     right_on=['Idestado','Idmunicipio'], how='left')
base_2020['Anio'] = 2020
base_2021 = pd.merge(df_inegi, df_prod_2021, 
                     left_on=['CVE_ENT', 'CVE_MUN'], 
                     right_on=['Idestado','Idmunicipio'], how='left')
base_2021['Anio'] = 2021
# base prod leche
base_leche_2019 = pd.merge(df_inegi, df_prod_leche[df_prod_leche['Anio']==2020], 
                     left_on=['NOM_ENT', 'NOM_MUN'], 
                     right_on=['Nomestado','Nommunicipio'], how='left')
base_leche_2019['Anio'] = 2019
base_leche_2019['Volumenproduccion'] = 0

base_leche_2020 = pd.merge(df_inegi, df_prod_leche[df_prod_leche['Anio']==2020], 
                     left_on=['NOM_ENT', 'NOM_MUN'], 
                     right_on=['Nomestado','Nommunicipio'], how='left')
base_leche_2020['Anio'] = 2020

base_leche_2021 = pd.merge(df_inegi, df_prod_leche[df_prod_leche['Anio']==2021], 
                     left_on=['NOM_ENT', 'NOM_MUN'], 
                     right_on=['Nomestado','Nommunicipio'], how='left')
base_leche_2021['Anio'] = 2021


base_2020.head()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,Anio,Idestado,Nomestado,Idmunicipio,Nommunicipio,Nomunidad,Nomcultivo,Volumenproduccion
0,1,Aguascalientes,1,Aguascalientes,2020,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Avena forrajera en verde,32571.0
1,1,Aguascalientes,1,Aguascalientes,2020,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Brócoli,295.6
2,1,Aguascalientes,1,Aguascalientes,2020,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Avena forrajera en verde,4352.0
3,1,Aguascalientes,1,Aguascalientes,2020,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Brócoli,306.6
4,1,Aguascalientes,1,Aguascalientes,2020,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Chile verde,313.5


In [100]:
df_prod = pd.concat([base_2019, base_2020, base_2021,
                     base_leche_2019, base_leche_2020, base_leche_2021], axis=0)

df_prod.head()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,Anio,Idestado,Nomestado,Idmunicipio,Nommunicipio,Nomunidad,Nomcultivo,Volumenproduccion
0,1,Aguascalientes,1,Aguascalientes,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Avena forrajera en verde,41182.48
1,1,Aguascalientes,1,Aguascalientes,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Cebada forrajera en verde,945.00
2,1,Aguascalientes,1,Aguascalientes,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Triticale forrajero en verde,12037.00
3,1,Aguascalientes,1,Aguascalientes,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Pastos y praderas,22808.00
4,1,Aguascalientes,1,Aguascalientes,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Avena forrajera en verde,4013.60


Filtramos los productos en el campo **Nomcultivo**:
- Frijol : Frijol, Frijol forrajero, Frijol x pelón, Semilla de frijol
- Arroz : Arroz palay
- Maíz : Maíz forrajero en verde, Maíz grano, Maíz palomero, Semilla de maíz grano
- Leche : Leche
- Trigo : Trigo forrajero verde, Trigo grano, Trigo ornamental, Semilla de trigo grano

In [101]:
prod_filt = {
    'Frijol': 'Frijol', 
    'Frijol forrajero': 'Frijol', 
    'Frijol x pelón': 'Frijol', 
    'Semilla de frijol': 'Frijol',
    'Arroz palay' : 'Arroz',
    'Maíz forrajero en verde': 'Maíz', 
    'Maíz grano': 'Maíz', 
    'Maíz palomero': 'Maíz', 
    'Semilla de maíz grano': 'Maíz',
    'Leche': 'Leche',
    'Trigo forrajero verde': 'Trigo', 
    'Trigo grano': 'Trigo', 
    'Trigo ornamental': 'Trigo', 
    'Semilla de trigo grano': 'Trigo'}

In [102]:
# se crea nuevo campo con nombres de productos únicos
df_prod['Producto'] = df_prod['Nomcultivo'].map(prod_filt)
# se filtra por llaves únicas 
df_prod = df_prod[df_prod['Producto'].isin(set(prod_filt.values()))]
df_prod.head()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,Anio,Idestado,Nomestado,Idmunicipio,Nommunicipio,Nomunidad,Nomcultivo,Volumenproduccion,Producto
8,1,Aguascalientes,1,Aguascalientes,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz forrajero en verde,244746.02,Maíz
9,1,Aguascalientes,1,Aguascalientes,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz grano,4151.04,Maíz
16,1,Aguascalientes,1,Aguascalientes,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Frijol,109.04,Frijol
17,1,Aguascalientes,1,Aguascalientes,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz forrajero en verde,48662.50,Maíz
18,1,Aguascalientes,1,Aguascalientes,2019,1.0,Aguascalientes,1.0,Aguascalientes,Tonelada,Maíz grano,624.34,Maíz


In [103]:
df_prod[df_prod['Nomcultivo']=='Frijol forrajero']

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,Anio,Idestado,Nomestado,Idmunicipio,Nommunicipio,Nomunidad,Nomcultivo,Volumenproduccion,Producto
3977715,16,Michoacán de Ocampo,99,Tuzantla,2019,16.0,Michoacán,99.0,Tuzantla,Tonelada,Frijol forrajero,14000.0,Frijol
3977738,16,Michoacán de Ocampo,99,Tuzantla,2019,16.0,Michoacán,99.0,Tuzantla,Tonelada,Frijol forrajero,14000.0,Frijol
3977761,16,Michoacán de Ocampo,99,Tuzantla,2019,16.0,Michoacán,99.0,Tuzantla,Tonelada,Frijol forrajero,14000.0,Frijol
3977784,16,Michoacán de Ocampo,99,Tuzantla,2019,16.0,Michoacán,99.0,Tuzantla,Tonelada,Frijol forrajero,14000.0,Frijol
3977807,16,Michoacán de Ocampo,99,Tuzantla,2019,16.0,Michoacán,99.0,Tuzantla,Tonelada,Frijol forrajero,14000.0,Frijol
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3983074,16,Michoacán de Ocampo,99,Tuzantla,2019,16.0,Michoacán,99.0,Tuzantla,Tonelada,Frijol forrajero,14000.0,Frijol
3983097,16,Michoacán de Ocampo,99,Tuzantla,2019,16.0,Michoacán,99.0,Tuzantla,Tonelada,Frijol forrajero,14000.0,Frijol
3983120,16,Michoacán de Ocampo,99,Tuzantla,2019,16.0,Michoacán,99.0,Tuzantla,Tonelada,Frijol forrajero,14000.0,Frijol
3983143,16,Michoacán de Ocampo,99,Tuzantla,2019,16.0,Michoacán,99.0,Tuzantla,Tonelada,Frijol forrajero,14000.0,Frijol


In [104]:
df_prod[df_prod['Nomcultivo']=='Leche']

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,Anio,Idestado,Nomestado,Idmunicipio,Nommunicipio,Nomunidad,Nomcultivo,Volumenproduccion,Producto
0,1,Aguascalientes,1,Aguascalientes,2019,NaN,Aguascalientes,NaN,Aguascalientes,Miles de lts,Leche,0.000,Leche
1,1,Aguascalientes,1,Aguascalientes,2019,NaN,Aguascalientes,NaN,Aguascalientes,Miles de lts,Leche,0.000,Leche
2,1,Aguascalientes,1,Aguascalientes,2019,NaN,Aguascalientes,NaN,Aguascalientes,Miles de lts,Leche,0.000,Leche
3,1,Aguascalientes,1,Aguascalientes,2019,NaN,Aguascalientes,NaN,Aguascalientes,Miles de lts,Leche,0.000,Leche
4,1,Aguascalientes,1,Aguascalientes,2019,NaN,Aguascalientes,NaN,Aguascalientes,Miles de lts,Leche,0.000,Leche
...,...,...,...,...,...,...,...,...,...,...,...,...,...
307734,32,Zacatecas,57,Trancoso,2021,NaN,Zacatecas,NaN,Trancoso,Miles de lts,Leche,1703.344,Leche
307735,32,Zacatecas,57,Trancoso,2021,NaN,Zacatecas,NaN,Trancoso,Miles de lts,Leche,1703.344,Leche
307736,32,Zacatecas,57,Trancoso,2021,NaN,Zacatecas,NaN,Trancoso,Miles de lts,Leche,1703.344,Leche
307737,32,Zacatecas,57,Trancoso,2021,NaN,Zacatecas,NaN,Trancoso,Miles de lts,Leche,1703.344,Leche


# Base a nivel entidad

In [105]:
df_entidad = pd.merge(df_prod, df_est.drop('country', axis=1),  
                      left_on=['NOM_ENT'], 
                      right_on=['state'], how='outer')

df_entidad.tail()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,Anio,Idestado,Nomestado,Idmunicipio,Nommunicipio,Nomunidad,Nomcultivo,Volumenproduccion,Producto,percentage,state
4775558,32,Zacatecas,57,Trancoso,2021,NaN,Zacatecas,NaN,Trancoso,Miles de lts,Leche,1703.344,Leche,0.462432,Zacatecas
4775559,32,Zacatecas,57,Trancoso,2021,NaN,Zacatecas,NaN,Trancoso,Miles de lts,Leche,1703.344,Leche,0.462432,Zacatecas
4775560,32,Zacatecas,57,Trancoso,2021,NaN,Zacatecas,NaN,Trancoso,Miles de lts,Leche,1703.344,Leche,0.462432,Zacatecas
4775561,32,Zacatecas,57,Trancoso,2021,NaN,Zacatecas,NaN,Trancoso,Miles de lts,Leche,1703.344,Leche,0.462432,Zacatecas
4775562,32,Zacatecas,57,Trancoso,2021,NaN,Zacatecas,NaN,Trancoso,Miles de lts,Leche,1703.344,Leche,0.462432,Zacatecas


In [107]:
df_states_reference = df_entidad.groupby(['Anio','NOM_ENT', 'Producto', 'Nomunidad']).agg({'Volumenproduccion': np.sum}).reset_index()
df_states_reference.columns = ['Anio', 'Entidad', 'Producto', 'Nomunidad', 'Volumenproduccion']

In [108]:
df_states_reference.tail()

,Anio,Entidad,Producto,Nomunidad,Volumenproduccion
383,2021,Yucatán,Maíz,Tonelada,7.436130e+07
384,2021,Zacatecas,Frijol,Tonelada,1.067818e+08
385,2021,Zacatecas,Leche,Miles de lts,3.350371e+07
386,2021,Zacatecas,Maíz,Tonelada,6.559402e+08
387,2021,Zacatecas,Trigo,Tonelada,5.429247e+06


In [109]:
df_states_reference[df_states_reference['Producto']=='Maíz']

,Anio,Entidad,Producto,Nomunidad,Volumenproduccion
2,2019,Aguascalientes,Maíz,Tonelada,4.339238e+08
4,2019,Baja California,Maíz,Tonelada,1.142208e+08
8,2019,Baja California Sur,Maíz,Tonelada,6.032397e+07
13,2019,Campeche,Maíz,Tonelada,1.434313e+08
17,2019,Chiapas,Maíz,Tonelada,7.039563e+08
...,...,...,...,...,...
370,2021,Tamaulipas,Maíz,Tonelada,1.921431e+08
374,2021,Tlaxcala,Maíz,Tonelada,4.606480e+07
379,2021,Veracruz de Ignacio de la Llave,Maíz,Tonelada,3.283825e+08
383,2021,Yucatán,Maíz,Tonelada,7.436130e+07


In [110]:
df_states_reference.to_csv('produccion_estados.csv', index=False,  encoding="utf-8-sig")